In [2]:


import os
import json
import boto3
import sklearn
import socket
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return "{}proxy/{}/jobs/".format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

# small fix to enable UI views
SparkContext.uiWebUrl = property(uiWebUrl)

# spark configurtion in local regime 
conf = SparkConf().set('spark.master', 'local[*]').set('spark.driver.memory', '8g')

#some needed objects
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

user: /user/st033822/


# Homework 1

Transform text file "The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft (Godwin) Shelle" into TF-IDF. Take sentence as "document".

### Part 1: 
- read text file as dataframe 
- filter out non-letters and empty strings 
- transform into dataframe doc_id -> tf_idf vector 


### Part 2:
- read text file as RDD
- filter out non-letters and empty strings 
- transform into rdd in format doc_id -> tf_idf vector


### NB(!) 

It's not allowed to use TF-IDF code from Spark internal libraries. 
It's not allowed to cast DF/RDD into pandas and use scikit-learn. Please, keep it spark. 


## Part 1

Reading text file as rdd

In [3]:


result_prefix = "malyutin_demo_hw1"

filepath = "file:///home/jovyan/shared/lectures_folder/84-0.txt"
from pyspark.sql.functions import monotonically_increasing_id

dataframe = sc.textFile(f"{filepath}")\
    .map(lambda x: (x,))\
    .toDF()\
    .select(F.col("_1").alias("text"))\
    .withColumn("id", monotonically_increasing_id())
dataframe.show()

+--------------------+---+
|                text| id|
+--------------------+---+
|The Project Guten...|  0|
|                    |  1|
|This eBook is for...|  2|
|most other parts ...|  3|
|whatsoever. You m...|  4|
|of the Project Gu...|  5|
|www.gutenberg.org...|  6|
|will have to chec...|  7|
|   using this eBook.|  8|
|                    |  9|
| Title: Frankenstein| 10|
|       or, The Mo...| 11|
|                    | 12|
|Author: Mary Woll...| 13|
|                    | 14|
|Release Date: 31,...| 15|
|[Most recently up...| 16|
|                    | 17|
|   Language: English| 18|
|                    | 19|
+--------------------+---+
only showing top 20 rows



In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import string
import re


def process_string(data):
    """
    basic preprocessing function:
    - removes punctuation
    - lower
    - split by space
    """
    punct_removed = re.sub(r'[^\w\s]','',data)
    words = punct_removed.lower().split(" ")
    
    
    return list(filter(lambda x: len(x) > 0, words))

# spark udf -- user defined function (~ mapper)

process_string_udf = udf(lambda z: process_string(z), ArrayType(StringType()))

In [246]:
"""
process words

filter out empty and small sentences
"""

by_words = dataframe\
    .select(process_string_udf(F.col("text")).alias("by_words"))\
    .where(F.size(F.col("by_words")) > 1)\
    .withColumn("doc_id", monotonically_increasing_id())
by_words.show()

+--------------------+------+
|            by_words|doc_id|
+--------------------+------+
|[the, project, gu...|     0|
|[this, ebook, is,...|     1|
|[most, other, par...|     2|
|[whatsoever, you,...|     3|
|[of, the, project...|     4|
|[wwwgutenbergorg,...|     5|
|[will, have, to, ...|     6|
|[using, this, ebook]|     7|
|[title, frankenst...|     8|
|[or, the, modern,...|     9|
|[author, mary, wo...|    10|
|[release, date, 3...|    11|
|[most, recently, ...|    12|
| [language, english]|    13|
|[character, set, ...|    14|
|[produced, by, ju...|    15|
|[further, correct...|    16|
|[start, of, the, ...|    17|
|[or, the, modern,...|    18|
|[by, mary, wollst...|    19|
+--------------------+------+
only showing top 20 rows



In [247]:
# exploding to get separate words
by_words_count = by_words.select(F.explode(F.col("by_words")).alias("word"), F.col("doc_id"))
by_words_count.show()

+--------------+------+
|          word|doc_id|
+--------------+------+
|           the|     0|
|       project|     0|
|     gutenberg|     0|
|         ebook|     0|
|            of|     0|
|  frankenstein|     0|
|            by|     0|
|          mary|     0|
|wollstonecraft|     0|
|        godwin|     0|
|       shelley|     0|
|          this|     1|
|         ebook|     1|
|            is|     1|
|           for|     1|
|           the|     1|
|           use|     1|
|            of|     1|
|        anyone|     1|
|      anywhere|     1|
+--------------+------+
only showing top 20 rows



In [248]:
# calculating term frequency

by_words_count = by_words_count.groupBy("doc_id", "word")\
    .agg(F.count("word").alias("tf"))
by_words_count.show()

+------+----------+---+
|doc_id|      word| tf|
+------+----------+---+
|    12|        13|  1|
|    62|      vain|  1|
|    92|expedition|  1|
|   127| fluctuate|  1|
|   170|   approve|  1|
|   171|      poor|  1|
|   189|        my|  1|
|   259|  remember|  1|
|   315|        in|  2|
|   319|     until|  1|
|   339|         i|  1|
|   419|  although|  1|
|   453|         a|  1|
|   477|        to|  1|
|   480|       you|  1|
|   506|        in|  1|
|   513|      what|  1|
|   515|  greatest|  1|
|   515|       but|  1|
|   536|  beaufort|  1|
+------+----------+---+
only showing top 20 rows



In [256]:
# document frequency

by_words_count_with_df = by_words_count.groupBy("word")\
      .agg(F.countDistinct("doc_id").alias("df"))
by_words_count_with_df.show()

+-------------+---+
|         word| df|
+-------------+---+
|         some|145|
|         hope| 49|
|        those| 91|
|          art|  7|
|    solemnity|  4|
|      blossom|  1|
|          few| 62|
|     ignominy|  4|
|        still| 65|
|       voyage| 15|
|    imitation|  2|
|       waters| 11|
|gratification|  3|
|    arguments|  7|
|      barrier|  4|
|   circulates|  1|
|apprehensions|  2|
|       spared|  5|
|     painters|  1|
|  transaction|  1|
+-------------+---+
only showing top 20 rows



In [286]:
# we use the same formula to calculate IDF that is included in Spark MLlib

import numpy as np
from pyspark.sql.functions import log, log1p
number_of_documents = dataframe.count()
import numpy as np
def calc_idf(df):
    """
    Args:
        df: document frequency - how many times a given word appears in whole corpus of docs (without normalization)
        
    Returns:
        idf: inverse document frequency
    """
    exp_idf = (number_of_documents + 1)/(df + 1) 
    idf = np.log(exp_idf)
    return idf

calc_idf_udf = udf(lambda df: calc_idf(docCount, df), FloatType())

In [283]:
import numpy as np
from pyspark.sql.functions import log, log1p
number_of_documents = dataframe.count()
import numpy as np
def calc_idf_exp(df):
    """
    Args:
        df: document frequency - how many times a given word appears in whole corpus of docs (without normalization)
        
    Returns:
        exp_idf: inverse document frequency without taking log
    """
    exp_idf = (number_of_documents + 1)/(df + 1) 

    return exp_idf

calc_idf_exp_udf = udf(lambda df: calc_idf(docCount, df), FloatType())

In [257]:
by_words_with_idf = by_words_count_with_df.withColumn("idf", calc_idf(F.col("df")))
by_words_with_idf.show()

+-------------+---+------------------+
|         word| df|               idf|
+-------------+---+------------------+
|         some|145|3.9710670072480765|
|         hope| 49| 5.042650623528266|
|        those| 91| 4.432885051907372|
|          art|  7| 6.875232087276577|
|    solemnity|  4| 7.345235716522312|
|      blossom|  1| 8.261526448396468|
|          few| 62|  4.81153890256488|
|     ignominy|  4| 7.345235716522312|
|        still| 65| 4.765018886929988|
|       voyage| 15| 6.182084906716632|
|    imitation|  2| 7.856061340288304|
|       waters| 11| 6.469766979168413|
|gratification|  3| 7.568379267836522|
|    arguments|  7| 6.875232087276577|
|      barrier|  4| 7.345235716522312|
|   circulates|  1| 8.261526448396468|
|apprehensions|  2| 7.856061340288304|
|       spared|  5| 7.162914159728358|
|     painters|  1| 8.261526448396468|
|  transaction|  1| 8.261526448396468|
+-------------+---+------------------+
only showing top 20 rows



In [258]:
tf_idf_full = by_words_with_idf.join(by_words_count, on=["word"], how = "left_outer")\
    .withColumn("tf_idf", F.col("tf") * F.col("idf"))
tf_idf_full.show()

+-------------+---+-----------------+----------+---+-----------------+
|         word| df|              idf|    doc_id| tf|           tf_idf|
+-------------+---+-----------------+----------+---+-----------------+
| accumulation|  2|7.856061340288304|      2614|  1|7.856061340288304|
| accumulation|  2|7.856061340288304|8589937145|  1|7.856061340288304|
|apprehensions|  2|7.856061340288304|       785|  1|7.856061340288304|
|apprehensions|  2|7.856061340288304|      1511|  1|7.856061340288304|
|    arguments|  7|6.875232087276577|8589935320|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|      1734|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|8589936445|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|      2676|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|      2327|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|       870|  1|6.875232087276577|
|    arguments|  7|6.875232087276577|       423|  1|6.875232087276577|
|     

In [259]:
tf_idf_aggregated = tf_idf_full.groupBy('doc_id')\
    .agg(F.collect_list('tf_idf').alias('tf_idf'))\
    .orderBy(F.col("doc_id"))
tf_idf_aggregated.show(truncate = 100)

+------+----------------------------------------------------------------------------------------------------+
|doc_id|                                                                                              tf_idf|
+------+----------------------------------------------------------------------------------------------------+
|     0|[2.7788063588506517, 7.568379267836522, 7.568379267836522, 4.466037259224273, 7.568379267836522, ...|
|     1|[6.652088535962367, 2.7281369596689475, 1.9273591149166367, 5.91015119123299, 3.231088527004032, ...|
|     2|[7.162914159728358, 2.410761783391621, 4.432885051907372, 7.568379267836522, 0.8581607114548204, ...|
|     3|[6.469766979168413, 2.644755350729896, 5.488937726156687, 5.553476247294258, 4.263325746727269, 7...|
|     4|[7.345235716522312, 2.410761783391621, 4.466037259224273, 0.8581607114548204, 7.345235716522312, ...|
|     5|[2.70084481738094, 5.289510701459792, 1.9273591149166367, 5.91015119123299, 6.875232087276577, 5....|
|     6|[3

In [285]:
# whole pipeline
by_words_count = sc.textFile(f"{filepath}")\
    .map(lambda x: (x,))\
    .toDF()\
    .select(F.col("_1").alias("text"))\
    .select(process_string_udf(F.col("text")).alias("by_words"))\
    .where(F.size(F.col("by_words")) > 1)\
    .withColumn("doc_id", monotonically_increasing_id())\
    .select(F.explode(F.col("by_words")).alias("word"), F.col("doc_id"))\
    .groupBy("doc_id", "word")\
    .agg(F.count("word").alias("tf"))

   

by_words_with_idf = by_words_count.groupBy("word")\
    .agg(F.countDistinct("doc_id").alias("df"))\
    .withColumn("idf", log(calc_idf_exp(F.col("df"))))
    

tf_idf = by_words_with_idf.join(by_words_count, on=["word"], how = "left_outer")\
    .withColumn("tf_idf", F.col("tf") * F.col("idf"))\
    .groupBy('doc_id')\
    .agg(F.collect_list('tf_idf').alias('tf_idf'))\
    .orderBy(F.col("doc_id"))

tf_idf.show(truncate = 100)

+------+----------------------------------------------------------------------------------------------------+
|doc_id|                                                                                              tf_idf|
+------+----------------------------------------------------------------------------------------------------+
|     0|[2.7788063588506517, 7.568379267836522, 7.568379267836522, 4.466037259224273, 7.568379267836522, ...|
|     1|[6.652088535962367, 2.7281369596689475, 1.9273591149166367, 5.91015119123299, 3.231088527004032, ...|
|     2|[7.162914159728358, 2.410761783391621, 4.432885051907372, 7.568379267836522, 0.8581607114548204, ...|
|     3|[6.469766979168413, 2.644755350729896, 5.488937726156687, 5.553476247294258, 4.263325746727269, 7...|
|     4|[7.345235716522312, 2.410761783391621, 4.466037259224273, 0.8581607114548204, 7.345235716522312, ...|
|     5|[2.70084481738094, 5.289510701459792, 1.9273591149166367, 5.91015119123299, 6.875232087276577, 5....|
|     6|[3

## Part 2

Reading text file into RDD

In [271]:
result_prefix = "malyutin_demo_hw1"

filepath = "file:///home/jovyan/shared/lectures_folder/84-0.txt"
from pyspark.sql.functions import monotonically_increasing_id

rddText = sc.textFile(f"{filepath}").repartition(1).zipWithIndex().repartition(5)



rddText.take(20)

[(' Letter 2', 40),
 (' Letter 3', 41),
 (' Letter 4', 42),
 (' Chapter 1', 43),
 (' Chapter 2', 44),
 (' Chapter 3', 45),
 (' Chapter 4', 46),
 (' Chapter 5', 47),
 (' Chapter 6', 48),
 (' Chapter 7', 49),
 ('Inspirited by this wind of promise, my daydreams become more fervent', 90),
 ('and vivid. I try in vain to be persuaded that the pole is the seat of', 91),
 ('frost and desolation; it ever presents itself to my imagination as the',
  92),
 ('region of beauty and delight. There, Margaret, the sun is for ever', 93),
 ('visible, its broad disk just skirting the horizon and diffusing a', 94),
 ('perpetual splendour. There—for with your leave, my sister, I will put', 95),
 ('some trust in preceding navigators—there snow and frost are banished;', 96),
 ('and, sailing over a calm sea, we may be wafted to a land surpassing in',
  97),
 ('wonders and in beauty every region hitherto discovered on the habitable',
  98),
 ('globe. Its productions and features may be without example, as the',

In [291]:
# tokenize RDD
rdd_tokenized = rddText.map(lambda line: process_string(line[0]))
rdd_tokenized.take(15)

[['letter', '2'],
 ['letter', '3'],
 ['letter', '4'],
 ['chapter', '1'],
 ['chapter', '2'],
 ['chapter', '3'],
 ['chapter', '4'],
 ['chapter', '5'],
 ['chapter', '6'],
 ['chapter', '7'],
 ['inspirited',
  'by',
  'this',
  'wind',
  'of',
  'promise',
  'my',
  'daydreams',
  'become',
  'more',
  'fervent'],
 ['and',
  'vivid',
  'i',
  'try',
  'in',
  'vain',
  'to',
  'be',
  'persuaded',
  'that',
  'the',
  'pole',
  'is',
  'the',
  'seat',
  'of'],
 ['frost',
  'and',
  'desolation',
  'it',
  'ever',
  'presents',
  'itself',
  'to',
  'my',
  'imagination',
  'as',
  'the'],
 ['region',
  'of',
  'beauty',
  'and',
  'delight',
  'there',
  'margaret',
  'the',
  'sun',
  'is',
  'for',
  'ever'],
 ['visible',
  'its',
  'broad',
  'disk',
  'just',
  'skirting',
  'the',
  'horizon',
  'and',
  'diffusing',
  'a']]

In [292]:
# rid of documents with length < 1 + indexing RDD
rdd_tokenized = rdd_tokenized.filter(lambda x: len(x) > 1).zipWithIndex()
rdd_tokenized.take(15)

[(['letter', '2'], 0),
 (['letter', '3'], 1),
 (['letter', '4'], 2),
 (['chapter', '1'], 3),
 (['chapter', '2'], 4),
 (['chapter', '3'], 5),
 (['chapter', '4'], 6),
 (['chapter', '5'], 7),
 (['chapter', '6'], 8),
 (['chapter', '7'], 9),
 (['inspirited',
   'by',
   'this',
   'wind',
   'of',
   'promise',
   'my',
   'daydreams',
   'become',
   'more',
   'fervent'],
  10),
 (['and',
   'vivid',
   'i',
   'try',
   'in',
   'vain',
   'to',
   'be',
   'persuaded',
   'that',
   'the',
   'pole',
   'is',
   'the',
   'seat',
   'of'],
  11),
 (['frost',
   'and',
   'desolation',
   'it',
   'ever',
   'presents',
   'itself',
   'to',
   'my',
   'imagination',
   'as',
   'the'],
  12),
 (['region',
   'of',
   'beauty',
   'and',
   'delight',
   'there',
   'margaret',
   'the',
   'sun',
   'is',
   'for',
   'ever'],
  13),
 (['visible',
   'its',
   'broad',
   'disk',
   'just',
   'skirting',
   'the',
   'horizon',
   'and',
   'diffusing',
   'a'],
  14)]

In [293]:
# document frequency - how many documents in the corpus contain the term

rdd_df = rdd_tokenized\
    .map(lambda line: (line[1], line[0]))\
    .flatMapValues(lambda line: line)\
    .map(lambda line: (line[1], line[0]))\
    .distinct()\
    .groupByKey()\
    .mapValues(len)
rdd_df.take(15)

[('presents', 3),
 ('is', 305),
 ('perpetual', 5),
 ('put', 15),
 ('on', 461),
 ('undertaking', 13),
 ('to', 1896),
 ('branches', 12),
 ('alarmed', 6),
 ('at', 318),
 ('would', 179),
 ('one', 197),
 ('us', 67),
 ('distant', 10),
 ('of', 2435)]

In [294]:
# Inverse document frequency
import numpy as np

from pyspark.sql.functions import log1p
rdd_idf = rdd_df\
    .map(lambda line: (line[0], calc_idf(line[1])))
rdd_idf.take(15)

[('presents', 7.568379267836522),
 ('is', 3.231088527004032),
 ('perpetual', 7.162914159728358),
 ('put', 6.182084906716632),
 ('on', 2.819108737874674),
 ('undertaking', 6.315616299341154),
 ('to', 1.4066446590213988),
 ('branches', 6.3897242714948765),
 ('alarmed', 7.008763479901099),
 ('at', 3.1894825261715685),
 ('would', 3.761716778066203),
 ('one', 3.6664065982618776),
 ('us', 4.735165923780306),
 ('distant', 6.556778356158042),
 ('of', 1.1565610001266253)]

In [295]:
# TERM FREQUENCY (tf)

rdd_tf = rdd_tokenized\
    .map(lambda line: (line[1], line[0]))\
    .flatMapValues(lambda line: line)\
    .groupBy(lambda line: line)\
    .mapValues(len)
rdd_tf.take(15)

[((0, 'letter'), 1),
 ((3, 'chapter'), 1),
 ((5, '3'), 1),
 ((6, 'chapter'), 1),
 ((7, 'chapter'), 1),
 ((7, '5'), 1),
 ((9, '7'), 1),
 ((10, 'of'), 1),
 ((11, 'vain'), 1),
 ((12, 'imagination'), 1),
 ((13, 'there'), 1),
 ((13, 'the'), 1),
 ((13, 'for'), 1),
 ((13, 'ever'), 1),
 ((14, 'its'), 1)]

In [277]:
# preparing RDD with calculated TF for joining

def mapper(rdd):
    return ( rdd[0][1], (rdd[0][0], rdd[1]))

rdd_tf_flat = rdd_tf.map(lambda x : mapper(x))

In [296]:
rdd_tf_flat.take(15)

[('letter', (0, 1)),
 ('chapter', (3, 1)),
 ('3', (5, 1)),
 ('chapter', (6, 1)),
 ('chapter', (7, 1)),
 ('5', (7, 1)),
 ('7', (9, 1)),
 ('of', (10, 1)),
 ('vain', (11, 1)),
 ('imagination', (12, 1)),
 ('there', (13, 1)),
 ('the', (13, 1)),
 ('for', (13, 1)),
 ('ever', (13, 1)),
 ('its', (14, 1))]

In [297]:
# join two RDD's with TF and with IDF by word

rdd_merged = rdd_tf_flat.join(rdd_idf)
rdd_merged.take(15)

[('of', ((10, 1), 1.1565610001266253)),
 ('of', ((25, 1), 1.1565610001266253)),
 ('of', ((33, 1), 1.1565610001266253)),
 ('of', ((45, 1), 1.1565610001266253)),
 ('of', ((49, 1), 1.1565610001266253)),
 ('of', ((53, 1), 1.1565610001266253)),
 ('of', ((73, 1), 1.1565610001266253)),
 ('of', ((77, 1), 1.1565610001266253)),
 ('of', ((93, 1), 1.1565610001266253)),
 ('of', ((134, 1), 1.1565610001266253)),
 ('of', ((136, 1), 1.1565610001266253)),
 ('of', ((148, 1), 1.1565610001266253)),
 ('of', ((185, 1), 1.1565610001266253)),
 ('of', ((192, 1), 1.1565610001266253)),
 ('of', ((198, 1), 1.1565610001266253))]

In [298]:
# calculating tf-idf for each word in each document (word itself is not represented)

def calculate_tf_idf(rdd):
    return (rdd[1][0][0], rdd[1][0][1] * rdd[1][1]) 

rdd_merged_tfidf = rdd_merged.map(lambda row: calculate_tf_idf(row))
rdd_merged_tfidf.take(15)

[(10, 1.1565610001266253),
 (25, 1.1565610001266253),
 (33, 1.1565610001266253),
 (45, 1.1565610001266253),
 (49, 1.1565610001266253),
 (53, 1.1565610001266253),
 (73, 1.1565610001266253),
 (77, 1.1565610001266253),
 (93, 1.1565610001266253),
 (134, 1.1565610001266253),
 (136, 1.1565610001266253),
 (148, 1.1565610001266253),
 (185, 1.1565610001266253),
 (192, 1.1565610001266253),
 (198, 1.1565610001266253)]

In [299]:
# final grouping by doc_id to get RDD "doc_id -> tf_idf vector"

rdd_merged_tfidf\
    .groupByKey()\
    .mapValues(list)\
    .sortBy(lambda x: x[0])\
    .take(15)

[(0, [7.162914159728358, 5.458166067489933]),
 (1, [5.458166067489933, 6.875232087276577]),
 (2, [5.458166067489933, 6.875232087276577]),
 (3, [7.008763479901099, 5.062853330845786]),
 (4, [7.162914159728358, 5.062853330845786]),
 (5, [5.062853330845786, 6.875232087276577]),
 (6, [6.875232087276577, 5.062853330845786]),
 (7, [7.568379267836522, 5.062853330845786]),
 (8, [7.856061340288304, 5.062853330845786]),
 (9, [7.856061340288304, 5.062853330845786]),
 (10,
  [1.1565610001266253,
   7.568379267836522,
   2.7788063588506517,
   2.861103858911277,
   5.428313104340251,
   7.856061340288304,
   1.6183879689351153,
   5.819179413027263,
   5.622469118781209,
   3.8670772937240288,
   7.856061340288304]),
 (11,
  [1.1565610001266253,
   3.231088527004032,
   1.0525560826799654,
   7.568379267836522,
   6.875232087276577,
   1.9273591149166367,
   3.065795670623532,
   1.1903776225058949,
   1.4066446590213988,
   7.345235716522312,
   7.345235716522312,
   6.315616299341154,
   1.716321